# StudyFlow Assistant (Colab Runner)
Run this notebook from your school Chromebook. Everything executes inside Colab's VM.

**First-time setup:** Run cells 1-4 once. After that, just run cell 5 to start.

In [ ]:
#@title 1. Download Project from GitHub
import os, shutil

if os.path.exists('/content/studyflow_assistant'):
    shutil.rmtree('/content/studyflow_assistant')

!git clone https://github.com/dfskaljf8/studyflow-assistant.git /content/studyflow_assistant
print('Project downloaded!')
print(os.listdir('/content/studyflow_assistant'))

In [ ]:
#@title 2. Install Dependencies
!pip install -q playwright==1.52.0 apscheduler==3.11.0 python-dotenv==1.1.0 tenacity==9.0.0 pydantic==2.11.3 pydantic-settings==2.9.1
!playwright install chromium
!playwright install-deps
print('All dependencies installed!')

In [ ]:
#@title 3. Set Up Environment (.env)
#@markdown Enter your Gemini API key below:
GEMINI_API_KEY = '' #@param {type:"string"}

env_path = '/content/studyflow_assistant/.env'
with open(env_path, 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
print(f'.env written')

if not GEMINI_API_KEY:
    print('WARNING: No API key set. Drafts will fail.')

In [ ]:
#@title 4. First-Time Login (Manual)
#@markdown This opens a visible browser via VNC so you can sign into Google Classroom.
#@markdown After running this cell, click the VNC link that appears, sign in, then stop this cell.

!apt-get update -qq && apt-get install -y -qq xvfb x11vnc > /dev/null 2>&1

import subprocess, os, time
subprocess.Popen(['Xvfb', ':99', '-screen', '0', '1280x720x24'])
time.sleep(1)
os.environ['DISPLAY'] = ':99'
subprocess.Popen(['x11vnc', '-display', ':99', '-forever', '-nopw', '-shared', '-rfbport', '5900'],
                  stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(1)

# Use colab's built-in port forwarding
from google.colab.output import eval_js
url = eval_js("google.colab.kernel.proxyPort(5900)")
print(f'\nOpen a VNC viewer or use this noVNC link to see the browser.')
print(f'If the link does not work, try installing the noVNC cell below.\n')

import asyncio
os.chdir('/content/studyflow_assistant')
import sys; sys.path.insert(0, '/content/studyflow_assistant')

async def manual_login():
    from playwright.async_api import async_playwright
    pw = await async_playwright().start()
    ctx = await pw.chromium.launch_persistent_context(
        user_data_dir='/content/studyflow_assistant/.browser_data',
        headless=False,
        args=['--no-sandbox', '--disable-dev-shm-usage'],
        viewport={'width': 1280, 'height': 720},
    )
    page = ctx.pages[0] if ctx.pages else await ctx.new_page()
    await page.goto('https://accounts.google.com/ServiceLogin')
    print('Browser opened to Google sign-in.')
    print('Sign in with your school Google account in the VNC window.')
    print('After signing in, go to classroom.google.com.')
    print('Once you see your classes, STOP this cell (square button).')
    while True:
        await asyncio.sleep(5)

try:
    await manual_login()
except (KeyboardInterrupt, asyncio.CancelledError):
    print('Login session saved!')

In [ ]:
#@title 5. Run StudyFlow (Single Run)
#@markdown Scans assignments, generates drafts, pastes into Google Docs.

import subprocess, os
os.chdir('/content/studyflow_assistant')
os.environ['DISPLAY'] = ':99'

result = subprocess.run(
    ['python', 'main.py', 'run'],
    capture_output=False,
    text=True,
    cwd='/content/studyflow_assistant',
    env={**os.environ, 'STUDYFLOW_MODE': 'run'},
)
print(f'\nExit code: {result.returncode}')

In [ ]:
#@title 6. (Optional) Save Session to Drive
#@markdown Saves browser session + drafts to Google Drive so they persist between Colab sessions.

from google.colab import drive
drive.mount('/content/drive')

import shutil, os
DRIVE_BACKUP = '/content/drive/MyDrive/studyflow_backup'
os.makedirs(DRIVE_BACKUP, exist_ok=True)

for folder in ['.browser_data', 'drafts', 'downloads']:
    src = f'/content/studyflow_assistant/{folder}'
    dst = f'{DRIVE_BACKUP}/{folder}'
    if os.path.exists(src):
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f'Saved {folder}')

print('Done! Session saved to Drive > studyflow_backup/')

In [ ]:
#@title 7. (Optional) Restore Session from Drive
#@markdown Run this AFTER cell 1 if you previously saved your session.

from google.colab import drive
drive.mount('/content/drive')

import shutil, os
DRIVE_BACKUP = '/content/drive/MyDrive/studyflow_backup'

for folder in ['.browser_data', 'drafts', 'downloads']:
    src = f'{DRIVE_BACKUP}/{folder}'
    dst = f'/content/studyflow_assistant/{folder}'
    if os.path.exists(src):
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f'Restored {folder}')

print('Session restored from Drive!')